# Project - Parameters with Highest Impact on House Prices


![Data Science Workflow](img/ds-workflow.png)

## Goal of Project
- A real estate dealer wants to figure out what matters most when selling a house
- They provide various sales data
- Your job is to figure out which 10 parameters (features) matter the most and present the findings

## Step 1: Acquire
- Explore problem
- Identify data
- Import data

### Step 1.a: Import libraries
- Execute the cell below (SHIFT + ENTER)
- NOTE: You might need to install mlxtend, if so, run the following in a cell
```
!pip install mlxtend
```

In [1]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

### Step 1.b: Read the data
- Use ```pd.read_parquet()``` to read the file `files/house_sales.parquet`
- NOTE: Remember to assign the result to a variable (e.g., ```data```)
- Apply ```.head()``` on the data to see all is as expected
    - The target is `SalePrice`

In [3]:
data = pd.read_csv('E://JOB//Power-BI-Training-Session-main//Day 1//New folder//Telco-Customer-Churn.csv')
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,2985,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [28]:
data['TotalCharges'].value_counts()/len(data)

140000    0.013699
135000    0.011644
155000    0.009589
145000    0.009589
190000    0.008904
            ...   
202665    0.000685
164900    0.000685
208300    0.000685
181500    0.000685
147500    0.000685
Name: SalePrice, Length: 663, dtype: float64

### Step 1.c: Inspect the data
- Check the number of rows and columns
    - HINT: `.shape`

In [29]:
data.shape

(1460, 56)

## Step 2: Prepare
- Explore data
- Visualize ideas
- Cleaning data

### Step 2.a: Check the data types
- This step tells you if some numeric column is not represented numeric.
- Get the data types by ```.dtypes```

In [30]:
data.dtypes

MSSubClass         int64
LotFrontage      float64
LotArea            int64
Street             int64
LotShape           int64
LandContour        int64
Utilities          int64
LandSlope          int64
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
MasVnrArea       float64
ExterQual          int64
ExterCond          int64
BsmtQual         float64
BsmtCond         float64
BsmtExposure     float64
BsmtFinType1     float64
BsmtFinSF1         int64
BsmtFinType2     float64
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
HeatingQC          int64
CentralAir         int64
1stFlrSF           int64
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BsmtFullBath       int64
BsmtHalfBath       int64
FullBath           int64
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
KitchenQual        int64
TotRmsAbvGrd       int64
Fireplaces         int64
FireplaceQu      float64


### Step 2.b: Check for null (missing) values
- Let's check if any features are not valuable
- Use ```.info()```
- Should we remove any?
    - You can remove features (columns):
    ```Python
data.drop([<column_name>, ..., <column_name>], axis=1)
```
- If you keep some with missing value you can add -1 `fillna(-1)`
    - Notice: This is not a validated or good approach - but for this purpose it will do

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 56 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   Street         1460 non-null   int64  
 4   LotShape       1460 non-null   int64  
 5   LandContour    1460 non-null   int64  
 6   Utilities      1460 non-null   int64  
 7   LandSlope      1460 non-null   int64  
 8   OverallQual    1460 non-null   int64  
 9   OverallCond    1460 non-null   int64  
 10  YearBuilt      1460 non-null   int64  
 11  YearRemodAdd   1460 non-null   int64  
 12  MasVnrArea     1452 non-null   float64
 13  ExterQual      1460 non-null   int64  
 14  ExterCond      1460 non-null   int64  
 15  BsmtQual       1423 non-null   float64
 16  BsmtCond       1423 non-null   float64
 17  BsmtExposure   1422 non-null   float64
 18  BsmtFinT

In [32]:
data = data.drop('PoolQC', axis=1)

In [33]:
data = data.fillna(-1)

## Step 3: Analyze
- Feature selection
- Model selection
- Analyze data

### Step 3.a: Quasi constant features
- Let see if there are any quasi features
- Create a `VarianceThreshold(threshold=0.01)` and fit it
- The features that are not quasi constant are given by `sel.get_feature_names_out()`
- Get all the qausi features as with list comprehension

In [34]:
sel = VarianceThreshold(threshold=0.01)
sel.fit(data)

VarianceThreshold(threshold=0.01)

In [35]:
len(sel.get_feature_names_out())

53

In [36]:
quasi_features = [col for col in data.columns if col not in sel.get_feature_names_out()]

In [37]:
quasi_features

['Street', 'Utilities']

### Step 3.b: Correlated features
- Calculate the correlation matrix `corr_matrix` and inspect it
    - HINT: use `.corr()`
- Get all the correlated features
    - HINT: A feature is correlated to a feature before it if
```Python
(corr_matrix[feature].iloc[:corr_matrix.columns.get_loc(feature)] > 0.8).any()
```
    - HINT: Use list comprehension to get a list of the correlated features

In [38]:
corr_matrix = data.corr()

In [39]:
corr_features = [feature for feature in corr_matrix.columns if (corr_matrix[feature].iloc[:corr_matrix.columns.get_loc(feature)] > 0.8).any()]

In [40]:
corr_features

['1stFlrSF',
 'TotRmsAbvGrd',
 'FireplaceQu',
 'GarageArea',
 'GarageQual',
 'GarageCond']

### Step 3.c: Prepare training and test set
- Assign all features in `X`
    - HINT: Use `.drop(['SalePrice'] + quasi_features + corr_features, axis=1)`
        - (assuming the same naming)
- Assign the target to `y`
    - HINT: The target is column `SalePrice`
- Split into train and test using `train_test_split`

In [41]:
X = data.drop(['SalePrice'] + quasi_features + corr_features, axis=1)
y = data['SalePrice']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Step 3.d: 10 best features for LinearRegression model
- Use the `SFS` to find 10 best features for a `LinearRegression` model
    - HINT: `SFS(LinearRegression(), k_features=10, verbose=2)`
    - HINT: when fitting fill missing values or remove them
        - Notice: ideally we would investigate them further to find appropriate values
- You get the best feature index from `.k_feature_idx_`

In [43]:
sfs = SFS(LinearRegression(), k_features=10, verbose=2)

In [44]:
sfs.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  46 out of  46 | elapsed:    0.9s finished

[2022-08-11 10:47:06] Features: 1/10 -- score: 0.6126893627288712[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.1s finished

[2022-08-11 10:47:07] Features: 2/10 -- score: 0.7190976519156257[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  44 out of  44 | elapsed:    0.9s finished

[2022-08-11 10:47:08] Features: 3/10 -- score: 0.7711947823299818[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

SequentialFeatureSelector(estimator=LinearRegression(), k_features=10,
                          verbose=2)

In [45]:
sfs.k_feature_idx_

(0, 2, 6, 10, 11, 15, 17, 26, 33, 36)

### Step 3.e: Test the result
- Create a normal `LinearRegression` model and run it on full data and calculate the r2_score
- Then do the same for only the 10 best features
- Did the score surprice you?
    - Notice that the test score is far from as good as from `SFS`

In [46]:
lin = LinearRegression()

In [47]:
lin.fit(X_train, y_train)
y_pred = lin.predict(X_test)
r2_score(y_test, y_pred)

0.7110172392141773

In [48]:
columns = X_train.columns[list(sfs.k_feature_idx_)]
lin.fit(X_train[columns], y_train)
y_pred = lin.predict(X_test[columns])
r2_score(y_test, y_pred)

0.7239540198372498

### Step 3.f: Test with 10 highest correlated features
- Find the 10 highest correlated features
    - HITN: `corr_matrix['SalePrice'].sort_values(ascending=False)`
- Then calculate the `r2_score` for them.
- Does the score surprice you?

In [49]:
columns = corr_matrix['SalePrice'].sort_values(ascending=False)[1:11].index

In [50]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('SalePrice', axis=1), y, random_state=0)

lin = LinearRegression()
lin.fit(X_train[columns], y_train)
y_pred = lin.predict(X_test[columns])
r2_score(y_test, y_pred)

0.7028207002269153

## Step 4: Report
- Present findings
- Visualize results
- Credibility counts

### Step 4.a: Present findings
- Use the analysis from Step 3 to figures out how to present your findings
- Try to think how the real estate dealer can use these findings

In [51]:
# Features that matter
X_train.columns[list(sfs.k_feature_idx_)]

Index(['MSSubClass', 'LotArea', 'Utilities', 'YearBuilt', 'YearRemodAdd',
       'BsmtQual', 'BsmtExposure', '1stFlrSF', 'HalfBath', 'KitchenQual'],
      dtype='object')

In [52]:
# Features correlated and removed
corr_features

['1stFlrSF',
 'TotRmsAbvGrd',
 'FireplaceQu',
 'GarageArea',
 'GarageQual',
 'GarageCond']

## Step 5: Actions
- Use insights
- Measure impact
- Main goal

### Step 5.a: Measure impact
- Can we help the dealer to use these insights?